### [Story](https://www.pivotaltracker.com/story/show/187949220)
### [Source](https://medium.com/@prathammodi001/keras-cnn-tutorial-classifying-images-made-easy-fb55cc8892ec)

Create from scratch a CNN apllied to computer vision, to classify cats and dogs from a well known image dataset published in [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/overview).

## Dog Breed Classifier CNN

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)
if tf.config.list_logical_devices('GPU'):
    print("Yes,  GPU available")
else:
    print("No GPU available")

### Dataset 

In [ ]:
# Load dataset
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.max.rows', None) # showw all rows

labels_csv = pd.read_csv("./dataset/labels.csv")
labels_csv.describe()

In [ ]:
# EDA (exploratory data analysis)
ax = labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

average_value = labels_csv["breed"].value_counts().mean()
ax.axhline(average_value, color="red", linestyle="--", label="Average")

plt.xlabel("Brred")
plt.ylabel("Count")
plt.title("Breed Distribution")
plt.legend


### Get images and their labels

In [ ]:
from IPython.display import Image

In [ ]:
# Test one image
for image in labels_csv["id"].head(1):
    display(Image(filename="./dataset/train/"+image+".jpg", width=300, height="300"))

In [ ]:
# get al images
filenames = []
for image_id in labels_csv["id"]:
    filenames.append("./dataset/train/"+image_id+".jpg")

filenames

In [ ]:
# check if all filenames of all images have been acquired
if len(os.listdir("./dataset/train")) == len(filenames):
    print("Correct! Filenames match amount of files!")
else:
    print("Error! Filenames do not match amount of files")

### labels

In [ ]:
labels = labels_csv["breed"].to_numpy()

labels

In [ ]:
# check for missing data, lables or images
if len(filenames) == len(labels):
    print("No missing Data!")
else:
    print("Missing data!!!")

In [ ]:
unique_breeds = labels
labels[0] == unique_breeds

boolean_labels = []

for label in labels:
    boolean_labels.append(label == unique_breeds)

## Prepare Train, Valid and Test sets

In [37]:
X = filenames
y = boolean_labels

NUM_IMAGES= 1000

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size=0.2, random_state=42)
len(X_train), len(y_train), len(X_val), len(y_val)

(800, 800, 200, 200)

## Images to tensors